In [1]:
#Install Dependencies
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
from google.colab import files
Train_Dataset = files.upload()

In [ ]:
!unzip Train.zip

In [ ]:
import os
from google.colab import files
Test_Dataset = files.upload()

In [ ]:
!unzip Test.zip

In [6]:
os.chdir("/content")
os.listdir()

['.config',
 'Train',
 'Test.zip',
 '__MACOSX',
 'Train.zip',
 'Test',
 'sample_data']

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Traingen = ImageDataGenerator(rescale=1./255)
Testgen = ImageDataGenerator(rescale=1./255)

train_generator = Traingen.flow_from_directory(
    '/content/Train',
    target_size=(256, 256),
    batch_size=65,
    class_mode='categorical',
    shuffle=True
)

test_generator = Testgen.flow_from_directory(
    '/content/Test',
    target_size=(256, 256),
    batch_size=65,
    shuffle=True,
    class_mode='categorical'
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.regularizers import l2

base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(256,256,3))

finetune_afterwards = 290

for layer in base_model.layers[:finetune_afterwards]:
    layer.trainable = False

GAP_2D = GlobalAveragePooling2D()
Dense1 = Dense(800, activation='relu', kernel_regularizer=l2(0.001))
Dropout1 = Dropout(0.2)
Dense2 = Dense(300, activation='relu', kernel_regularizer=l2(0.001))
Dropout2 = Dropout(0.2)
prediction = Dense(4, activation='softmax')

model = Sequential([
    base_model,
    GAP_2D,
    Dense1,
    Dropout1,
    Dense2,
    Dropout2,
    prediction
])


In [14]:
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping
import pickle

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='test_accuracy', mode='max', patience=2,  restore_best_weights=True)

STEP_SIZE_TRAIN = 85
STEP_SIZE_TEST = 20


run = model.fit(train_generator,
                steps_per_epoch = STEP_SIZE_TRAIN,
                validation_data = test_generator,
                validation_steps= STEP_SIZE_TEST,
                epochs=7
)

pickle.dump(model, open('model.pkl', 'wb'))

Epoch 1/7
85/85 [==============================] - 1172s 14s/step - loss: 0.6400 - accuracy: 0.9172 - val_loss: 0.4595 - val_accuracy: 0.9400
Epoch 2/7
85/85 [==============================] - 1139s 13s/step - loss: 0.3483 - accuracy: 0.9616 - val_loss: 0.3780 - val_accuracy: 0.9392
Epoch 3/7
85/85 [==============================] - 1164s 14s/step - loss: 0.2288 - accuracy: 0.9772 - val_loss: 0.2978 - val_accuracy: 0.9523
Epoch 4/7
85/85 [==============================] - 1209s 14s/step - loss: 0.1871 - accuracy: 0.9773 - val_loss: 0.2903 - val_accuracy: 0.9423
Epoch 5/7
85/85 [==============================] - 1208s 14s/step - loss: 0.1472 - accuracy: 0.9819 - val_loss: 0.2373 - val_accuracy: 0.9562
Epoch 6/7
85/85 [==============================] - 1306s 15s/step - loss: 0.1229 - accuracy: 0.9859 - val_loss: 0.1878 - val_accuracy: 0.9592
Epoch 7/7
85/85 [==============================] - 1184s 14s/step - loss: 0.0914 - accuracy: 0.9926 - val_loss: 0.1658 - val_accuracy: 0.9692


In [15]:
scores = model.evaluate(test_generator, steps=STEP_SIZE_TEST)
print(scores)

20/20 [==============================] - 210s 10s/step - loss: 0.1647 - accuracy: 0.9700
[0.1646844893693924, 0.9700000286102295]
